# Numerical Root Finding
---
- Author: Diego Inácio
- GitHub: [github.com/diegoinacio](https://github.com/diegoinacio)
- Notebook: [root-finding.ipynb](https://github.com/diegoinacio/computer-vision-notebooks/blob/master/Mathematical-Foundations/root-finding.ipynb)
---
Overview and implementation of some numerical methods for *root finding*.

<font color="#CC0000">[<b>PT-BR</b> content]</font>

In [1]:
import numpy as np

Na matemática, a raiz (ou "*zero*") de uma função polinomial $f(x)$ é todo valor de $x$ em que a mesma função intercepta o eixo das abscissas no plano cartesiano, ou seja: $f(x)=0$. Geralmente encontramos os valores das raízes de forma analítica (ou *método direto*). Por exemplo, dada uma função do primeiro grau $f(x)=2x-2$, podemos obter a equação $2x-2=0$ e determinar que a raíz é $1$, já que:

$$
f(1)=2 \cdot 1-2=2-2=0
$$

Se a função for do segundo grau, é possível encontrar (ou não) suas raízes por meio de [fatoração](https://pt.wikipedia.org/wiki/Fatora%C3%A7%C3%A3o) ou pela [fórmula de Bhaskara](https://pt.wikipedia.org/wiki/Bhaskara_II#A_f%C3%B3rmula_para_encontrar_as_ra%C3%ADzes_da_equa%C3%A7%C3%A3o_quadr%C3%A1tica). Uma função quadrática pode ter até 2 raízes. Ou seja, em polinômios, o número de raízes possíveis está ligado ao grau ou *ordem* de uma função. Uma função do segundo grau é um polinômio de grau 2 e, desta forma, pode possuir até 2 raízes. Um polinômio de grau 3 terá até 3 raízes, um de 4 terá 4 e assim por diante. Nem sempre é fácil encontrar o valor de uma raiz de forma analítica, pois elas podem não ser exatas. Para isso são utilizados métodos numéricos, que visam encontrar o valor da raiz por meio de convergência. Para os exemplos a seguir, utilizaremos a função:

$$ \large
f(x)=\frac{x^3}{4}+\frac{3x^2}{4}-\frac{3x}{2}-2
$$

In [2]:
# definição da função f(x)
f = lambda x: x**3/4 + 3*x**2/4 - 3*x/2 - 2
# raízes adquiridas de forma analítica
raizes = [-4, -1, 2]

Apesar de a função ser de facil solução analítica ou seja, suas raízes serem fáceis de serem encontradas de forma direta, iremos utilizá-la para fins de estudo e verificação do funcionamento dos métodos. A função possui 3 raízes reais, como podem ser verificados a seguir.

![root-finding analytical](output/root-finding_analytical.png "Root-finding Analytical")

## 1. Método da bissecão
---
O [método da bisseção](https://pt.wikipedia.org/wiki/M%C3%A9todo_da_bisse%C3%A7%C3%A3o) funciona com o particionamento de um intervalo inicial $[a,b]$ e seguido da seleção do subintervalo que contenha a raiz, tornando-o o próximo intervalo a ser bisseccionado. Os valores para **a** e **b** serão redefinidos com as extremidades do subintervalo e o processo irá se repetir até que o valor de uma das extremidades aplicado à função ou tamanho do intervalo seja menor que o parâmetro de erro $\large \epsilon$. A bisseção é determinada pelo ponto médio entre o intervalo, de forma que:

$$ \large
x_n=\frac{a+b}{2}, \quad \textrm{para} \quad n=1,2,3 \ldots
$$

O intervalo $[a,b]$ será válido apenas se houver apenas um número ímpar de raízes entre ele, caso contrário o algorítmo não irá convergir. A verificação da existência de uma raiz entre um intervalo qualquer é dada com a verificação da troca de sinais entre o mesmo, de forma que:

$$ \large
f(a) \cdot f(b) \geq 0
$$

Caso intervalo seja válido, a seleção do subintervalo particionado é dada por:

$$
\textrm{Se} \quad f(a) \cdot f(x_n) < 0, \quad \textrm{então} \quad b=x_n \quad \textrm{senão} \quad a=x_n
$$

In [3]:
%%time
# parâmetros de entrada
e = 1e-3     # erro
a, b = 0, 3  # definição do intervalor [a,b]
# testes de interválido válido
if a > b: raise Exception('Intervalo inválido! "b" deve ser maior do que "a"')
if f(a)*f(b) >= 0: raise Exception('Intervalo inválido! Não existe raiz ou existem múltiplas raízes entre "a" e "b"')
# método da bisseção
xn = np.array([a, b, (a + b)/2])
while (abs(f(xn[-1])) or abs(b - a)) > e:
    if f(a)*f(xn[-1]) < 0:
        b = xn[-1]
    else:
        a = xn[-1]
    xn = np.append(xn, (a + b)/2)
print('iterações:', len(xn) - 2)
print(xn)

iterações: 13
[0.         3.         1.5        2.25       1.875      2.0625
 1.96875    2.015625   1.9921875  2.00390625 1.99804688 2.00097656
 1.99951172 2.00024414 1.99987793]
Wall time: 3.49 ms


![root-finding bisection method](output/root-finding_bisection.gif "Root-finding Bisection Method")

## 2. Método da falsa posição
---
O [método da falsa posição](https://pt.wikipedia.org/wiki/M%C3%A9todo_da_posi%C3%A7%C3%A3o_falsa) é bastante semelhante ao da *bisseção*. Ele particiona o intervalo e seleciona subintervalos, até que o um intervalo selecionado tenha o valor de uma das extremidades menor que o erro $\large \epsilon$. A diferença é que, na bisseção, o particionamento do intervalo era dado por uma média aritimética simples. Já no método da *falsa posição* a média será ponderada com base na extremidade que tiver valor mais próximo à raiz. Ou seja, a extremidade que tiver o menor valor. A média aritimética ponderada pode ser entendida por:

$$ \large
x_n=a-\frac{(b-a) \cdot f(a)}{f(b)-f(a)}, \quad \textrm{para} \quad n=1,2,3 \ldots
$$

Note que a extremidade que irá convergir será a de menor valor e não necessariamente a mais próxima, assim como acontece no exemplo a seguir. O critério de escolha da extremidade pode ser entendido por:

$$
\textrm{Se} \quad f(a) \cdot f(x_n) < 0, \quad \textrm{então} \quad b=x_n \quad \textrm{senão} \quad a=x_n
$$

In [4]:
%%time
# parâmetros de entrada
e = 1e-3     # erro
a, b = 0, 3  # definição do intervalor [a,b]
# testes de interválido válido
if a > b: raise Exception('Intervalo inválido! "b" deve ser maior do que "a"')
if f(a)*f(b) >= 0: raise Exception('Intervalo inválido! Não existe raiz ou existem múltiplas raízes entre "a" e "b"')
# método da falsa posição
xn = np.array([a, b, a - f(a)*(b - a)/(f(b) - f(a))])
while abs(f(xn[-1])) > e:
    if f(a)*f(xn[-1]) < 0:
        b = xn[-1]
    else:
        a = xn[-1]
    xn = np.append(xn, a - f(a)*(b - a)/(f(b) - f(a)))
print('iterações:', len(xn) - 2)
print(xn)

iterações: 11
[0.         3.         0.66666667 1.2972973  1.69565809 1.88220005
 1.95662066 1.98433291 1.99438199 1.99799067 1.99928201 1.99974353
 1.9999084 ]
Wall time: 1.5 ms


![root-finding false position method](output/root-finding_false_position.gif "Root-finding False Position Method")

## 3. Método Newton-Raphson
---
O [método newton-raphson](https://pt.wikipedia.org/wiki/M%C3%A9todo_de_Newton%E2%80%93Raphson) (ou *método das tangentes*) é sem dúvida o método que converge mais rápido, porém existe um preço a se pagar por isso. O primeiro, assim como nos outros métodos, é necessário saber se no intervalo existe uma raiz. O segundo é que, para se obter a tangente da função, deve-se ter o conhecimento analítico da *primeira derivada* de $f(x)$, de forma que:

$$ \large
x_n=x_{n-1}-\frac{f(x_{n-1})}{f'(x_{n-1})}, \quad \textrm{para} \quad n=1,2,3 \ldots
$$

Um passo bastante importante para o bom funcionamento do método é escolher a melhor extremidade do intervalo $[a,b]$ para iniciar o processo. Para isso, faremos uso da *segunda derivada*, de forma que:

$$
f(x_i) \cdot f''(x_i) > 0, \quad \textrm{Para} \quad i=\{ \textrm{Extremos do intervalo} \}
$$

A representação de $f'(x)$ e $f''(x)$ podem ser entendidas, respectivamente:

$$
f'(x)=\frac{d}{dx}f(x)=\frac{3x^2}{4}+\frac{3x}{2}-\frac{3}{2} \quad ; \quad f''(x)=\frac{d}{dx}f'(x)=\frac{d}{dx}\frac{d}{dx}f(x)=\frac{3x}{2}+\frac{3}{2}
$$

In [5]:
df = lambda x: 3*x**2/4 + 3*x/2 - 3/2  # primeira derivada de f(x) 
ddf = lambda x: 3*x/2 + 3/2            # segunda derivada de f(x)

In [6]:
%%time
# parâmetros de entrada
e = 1e-3     # erro
a, b = 0, 3  # definição do intervalor [a,b]
# testes de interválido válido
if a > b: raise Exception('Intervalo inválido! "b" deve ser maior do que "a"')
if f(a)*f(b) >= 0: raise Exception('Intervalo inválido! Não existe raiz ou existem múltiplas raízes entre "a" e "b"')
# definição do melhor extremo do intervalo
# xi é equivalente à x_{n-1}
if f(a)*ddf(a) > 0:
    xi = a
elif f(b)*ddf(b) > 0:
    xi = b
else:
    raise Exception('Intervalo inválido! Defina outro intervalo mais preciso')
# método newton-raphson
xn = np.array([a, b, xi - f(xi)/df(xi)])
while abs(xn[-1] - xi) > e:
    xi = xn[-1]
    xn = np.append(xn, xi - f(xi)/df(xi))
print('iterações:', len(xn) - 2)
print(xn)

iterações: 4
[0.         3.         2.28205128 2.03263278 2.00051927 2.00000013]
Wall time: 998 µs


![root-finding newton-raphson method](output/root-finding_newton-raphson.gif "Root-finding Newton-Raphson Method")

## 4. Método das secantes
---
O [método das secantes](https://pt.wikipedia.org/wiki/M%C3%A9todo_das_secantes) é uma opção que geralmente converge mais rapido que o da *bisseção* e não necessita da aquisição analítica da derivada, assim como no *newton-raphson*. Ao invés disso, semelhantemente ao método da *falsa posição*, é utilizado o quociente da diferença no lugar de $f'(x)$, de forma que:

$$ \large
x_{n+1}=x_n-\frac{f(x_n)}{\frac{f(x_n)-f(x_{n-1})}{x_n-x_{n-1}}}=x_n-\frac{(x_n-x_{n-1}) \cdot f(x_n)}{f(x_n)-f(x_{n-1})}, \quad \textrm{para} \quad n=1,2,3 \ldots
$$

In [7]:
%%time
# parâmetros de entrada
e = 1e-3     # erro
a, b = 0, 3  # definição do intervalor [a,b]
# testes de interválido válido
if a > b: raise Exception('Intervalo inválido! "b" deve ser maior do que "a"')
if f(a)*f(b) >= 0: raise Exception('Intervalo inválido! Não existe raiz ou existem múltiplas raízes entre "a" e "b"')
# método das secantes
# xo = x_{n-1}
# xm = x_n
# xi = x_{n+1}
xo = a; xm = b
xn = np.array([xo])
while abs(f(xn[-1])) > e:
    xi = xm - (xm - xo)*f(xm)/(f(xm) - f(xo))
    xn = np.append(xn, xi)
    xo = xi
print('iterações:', len(xn) - 2)
print(xn)

iterações: 10
[0.         0.66666667 1.2972973  1.69565809 1.88220005 1.95662066
 1.98433291 1.99438199 1.99799067 1.99928201 1.99974353 1.9999084 ]
Wall time: 4.49 ms


![root-finding secant method](output/root-finding_secant.gif "Root-finding Secant Method")